In [1]:
# Uncomment these lines if you have not installed any packages yet
# !pip install vislearnlabpy

In [2]:
import os
from pathlib import Path

from PIL import Image
import pymongo as pm
from dotenv import load_dotenv
load_dotenv()
from vislearnlabpy.extractions.drawings import MongoExtractor

Fill out the fields below. Copy the .env_template file to a .env file and enter your username and password.

In [3]:
# make sure to add password to a .env file
DATABASE_NAME = "kiddraw"
COLLECTION_NAME = "birch_run_v1"
SAVE_DIR = Path(os.getcwd()) / "mongo_output" # often set to /Volumes/vislearnlab/experiments/drawings/data..

This is the Mongo extractor

In [4]:
extractor = MongoExtractor(os.getenv("MONGO_CONNECTION"), DATABASE_NAME, COLLECTION_NAME, SAVE_DIR)

Can be used to directly prompt Mongo with PyMongo

In [ ]:
all_images = list(extractor.collection.find({'$and': [{'dataType':'finalImage'}]}))
print(f"Total final images in {extractor.collection.name}: {len(all_images)}")

Or can extract full datasets as needed.

In [ ]:
extractor.extract_images(transform_file=True)

In [ ]:
extractor.extract_audio()

Extracting strokes!

In [5]:
shark_strokes = list(extractor.collection
                     
# sessionId is the unique identifier for each participant session
                .find({#"sessionId": session_id,
                    
                    "dataType": "stroke",

# Trial number is the index of the trial of the current participant
                    #"trialNum": trial_num,

# Categories usually include articles, you can find all the categories here: (https://github.com/brialorelle/museumkiosk/blob/83fff9902c2cdd4369540c1feed5c3a1be1c655f/code/experiment/trial-sequence.js#L37)
                    "category": "a shark"
                    }).sort("startTrialTime"))

These are the SVG values for each stroke that can be rendered as png files etc.

In [ ]:
[s['svg'] for s in shark_strokes]